In [3]:
"""
我们知道，只要实现__len__和__getitem__方法，类可以表现得像一个序列
"""
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self,components):
        self._components = array(self.typecode,components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self.components) # 该函数返回的是array('d',[0.0,1.0,....])
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + btyes(self._components))
    
    def __eq__(self,other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self,index):
        return self._components[index]
    
    @classmethod
    def frombytes(cls,octets):
        typecode = chr(cotets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
v1 = Vector([3,4,5])
print(len(v1))
print(v1[-1])
print(v1[1:3])

3
5.0
array('d', [4.0, 5.0])


In [4]:
"""
上面的类有个问题，Vector实例的切片是array，如果是Vector的话，那就更好了。
不过在此之前，我们先了解一下切片的原理
"""
class MySeq:
    def __getitem__(self,index):
        return index
    
s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2,9]) # 如果[]中有逗号，__getitem__收到的是元组
print(s[1:4:2,7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [5]:
"""
接下来，我们来看一下slice本身的属性,可以看到，它有start,stop和step数据属性，以及indices方法
"""
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [6]:
"""
indices方法如下：
S.indices(len) -> (start,stop,stride)
解释起来比较麻烦，咱们直接看代码.
假设序列为'ABCDE'
第一个的结果是(0, 5, 2)，也就是说，给定一个长度为5的序列，'ABCDE'[:10:2] 等同于 'ABCDE'[0:5:2]
第二个结果为（2,5,1）也就是说'ABCDE'[-3:] 等同于 'ABCDE'[2:5:1]
"""
print(slice(None,10,2).indices(5))
print(slice(-3,None,None).indices(5))


(0, 5, 2)
(2, 5, 1)


In [9]:
"""
在了解了切片原理之后，我们修改之前的__getitem__，使得切片操作返回的仍然是一个Vector实例
"""
import numbers
class Vector:
    typecode = 'd'
    
    def __init__(self,components):
        self._components = array(self.typecode,components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self.components) # 该函数返回的是array('d',[0.0,1.0,....])
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + btyes(self._components))
    
    def __eq__(self,other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self,index):
        cls = type(self)
        if isinstance(index,slice):
            return cls(self._components[index])
        elif isinstance(index,numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    @classmethod
    def frombytes(cls,octets):
        typecode = chr(cotets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
v1 = Vector([3,4,5])
print(len(v1))
print(v1[-1])
print(type(v1[0:2]))

3
5.0
<class '__main__.Vector'>
